<a href="https://colab.research.google.com/github/hayleypring1/conjoint-example/blob/master/Indonesia_google_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytrends





In [3]:
import pandas as pd
from pytrends.request import TrendReq
from datetime import datetime, timedelta
import time
import os

pytrends = TrendReq(hl='en-US', tz=360)
kw_list = ["perubahan iklim"]

# Define the time frame
start_date = datetime(2010, 1, 1)
end_date = datetime(2010, 2, 1)

# Create a list of intervals covering every month
intervals = []
current_date = start_date
while current_date <= end_date:
    current_month_end = current_date + pd.offsets.MonthEnd(0)
    intervals.append((current_date, current_month_end))
    current_date = current_month_end + timedelta(days=1)

print(intervals)

[(datetime.datetime(2010, 1, 1, 0, 0), Timestamp('2010-01-31 00:00:00')), (Timestamp('2010-02-01 00:00:00'), Timestamp('2010-02-28 00:00:00'))]


In [5]:
# Initialize an empty DataFrame to store all results
all_results_df = pd.DataFrame()

# Iterate through each interval
for interval in intervals:
    start_interval, end_interval = interval

    pytrends.build_payload(kw_list, cat=0, timeframe=f'{start_interval.strftime("%Y-%m-%d")} {end_interval.strftime("%Y-%m-%d")}', geo='ID', gprop='')

    # Assuming you want to store the interest by region data in a DataFrame
    interest_by_region_df = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)

    # Reset index to include region names in the DataFrame
    interest_by_region_df.reset_index(inplace=True)

    # Add a column for the month searched
    interest_by_region_df['Month_Searched'] = start_interval.strftime("%Y-%m")

    # Export the DataFrame to a CSV file
    csv_filename = f'interest_by_region_data_{start_interval.strftime("%Y-%m")}.csv'
    interest_by_region_df.to_csv(csv_filename, index=False)

    # Append the results to the main DataFrame
    all_results_df = pd.concat([all_results_df, interest_by_region_df], ignore_index=True)

    # Sleep for 3 seconds to avoid rate-limiting
    time.sleep(25)

# Export the combined DataFrame to a CSV file
all_results_df.to_csv('combined_interest_by_region_data.csv', index=False)

print(pytrends)

TooManyRequestsError: The request failed: Google returned a response with code 429

In [ ]:
#now cities

# Google Trend decoder
import pandas as pd
from pytrends.request import TrendReq
from datetime import datetime, timedelta
import time

pytrends = TrendReq(hl='en-US', tz=360)
kw_list = ["mudanças climáticas"]

# Define the time frame
start_date = datetime(2018, 2, 1)
end_date = datetime(2020, 1, 31)

#do DF because that's missing...

# List of the Provinces in Indonesia
Indonesia_provinces = [
    "ID-AC", "ID-BA", "ID-BB", "ID-BT", "ID-BE", "ID-JT", "ID-KT", "ID-PT",
    "ID-ST", "ID-JI", "ID-KI", "	ID-NT", "	ID-GO", "ID-PE", "ID-JK", "ID-JA",
    "ID-LA", "ID-MA", "ID-KU", "ID-MU", "ID-SA", "ID-SU", "ID-PA", "ID-RI",
    "ID-KR", "ID-SG", "ID-KS", "ID-PS", "ID-SN", "ID-SS", "ID-PD", "ID-JB",
    "ID-KB", "ID-NB", "ID-PB", "ID-SR", "ID-SB", "ID-YO"
]

# Create a list of intervals covering every month
intervals = []
current_date = start_date
while current_date <= end_date:
    current_month_end = current_date + pd.offsets.MonthEnd(0)
    intervals.append((current_date, current_month_end))
    current_date = current_month_end + timedelta(days=1)

# Iterate through each interval
for interval in intervals:
    start_interval, end_interval = interval

    for state in Indonesia_provinces:
        try:
            pytrends.build_payload(kw_list, cat=0, timeframe=f'{start_interval.strftime("%Y-%m-%d")} {end_interval.strftime("%Y-%m-%d")}', geo=state, gprop='')

            # Store the interest by city data in a DataFrame
            interest_by_city_df = pytrends.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=False)

            # Reset index to include city names in the DataFrame
            interest_by_city_df.reset_index(inplace=True)

            # Add columns for the month searched and state
            interest_by_city_df['Month_Searched'] = start_interval.strftime("%Y-%m")
            interest_by_city_df['State'] = state

            # Export the DataFrame to a CSV file
            csv_filename = f'interest_by_city_data_{state}_{start_interval.strftime("%Y%m")}.csv'
            interest_by_city_df.to_csv(csv_filename, index=False)

            print(f"Processed: {state} - {start_interval.strftime('%Y-%m')}")
        except Exception as e:
            print(f"Error processing {state} - {start_interval.strftime('%Y-%m')}: {e}")

        # Sleep for 3 seconds to avoid rate-limiting
        time.sleep(3)

print("done")